# Tensorflow intro

This tutorial shows the basic usage of tensorflow to train neural networks

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import datetime

In [2]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification

# Simple NN for classification

## Data processing
Read data and convert them to numerical inputs

In [3]:
X, y = make_classification(n_samples=20000, n_features=8, n_informative=5, 
                           n_redundant=0, n_classes=2, random_state=1)

In [4]:
X, X_test, y, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [5]:
X_train, X_dev, y_train, y_dev = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [6]:
print('train size: {}, target_ratio: {:.3f}'.format(X_train.shape, np.mean(y_train)))
print('test size: {}, target_ratio: {:.3f}'.format(X_test.shape, np.mean(y_test)))
print('dev size: {}, target_ratio: {:.3f}'.format(X_dev.shape, np.mean(y_dev)))

train size: (12800, 8), target_ratio: 0.502
test size: (4000, 8), target_ratio: 0.501
dev size: (3200, 8), target_ratio: 0.502


## Building a simple model with tf.keras

Very useful documentations with many examples and detailed explanation of everything you might need:
 - https://www.tensorflow.org/api_docs/python/tf/keras/
 - https://keras.io/api/

Contain everything about:
  - Model building: Activations, Losses, Optimizers, Regularization
  - Data processing
  - Pretrained models and datasets
  - Automatic differentiation
  - ...

  

### Model speficication

three APIs for building the model
   - sequential - easy to code, but less flexible - we will use it sometimes
   - functional - flexible and still easy to code - we will use it the most
   - model subclassing - rather complicated and not very much used - we will skip it

#### Sequential API

(https://www.tensorflow.org/guide/keras/sequential_model)

Easy to code but <span style="color:red"> NOT </span> appropriate when:

- Your model has multiple inputs or multiple outputs
- Any of your layers has multiple inputs or multiple outputs
- You need to do layer sharing
- You want non-linear topology (e.g. a residual connection, a multi-branch model)

In [7]:
# Specification A)

model = tf.keras.Sequential([
    tf.keras.layers.InputLayer([X_train.shape[1],]), # Create input layer with 'input data' neurons
    tf.keras.layers.Dense(10, activation="relu"), # Create hidden layer with 10 neurons and ReLU activation
    tf.keras.layers.Dense(1, activation="sigmoid"), # Create output layer with one neuron and sigmoid activation
])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 10)                90        
                                                                 
 dense_1 (Dense)             (None, 1)                 11        
                                                                 
Total params: 101
Trainable params: 101
Non-trainable params: 0
_________________________________________________________________


In [8]:
# Specification B)

model = tf.keras.Sequential()
model.add(tf.keras.Input(shape=(X_train.shape[1],)))
model.add(tf.keras.layers.Dense(10, activation="relu"))
model.add(tf.keras.layers.Dense(1, activation="sigmoid"))

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 10)                90        
                                                                 
 dense_3 (Dense)             (None, 1)                 11        
                                                                 
Total params: 101
Trainable params: 101
Non-trainable params: 0
_________________________________________________________________


#### Functional API

(https://www.tensorflow.org/guide/keras/functional)

The Keras functional API is a way to create models that are more flexible than the tf.keras.Sequential API. The functional API can handle models with non-linear topology, shared layers, and even multiple inputs or outputs.

The main idea is that a deep learning model is usually a directed acyclic graph (DAG) of layers. So the functional API is a way to build graphs of layers.



In [9]:
inputs = tf.keras.Input(shape=(X_train.shape[1],))

hidden = tf.keras.layers.Dense(10)(inputs)
hidden = tf.keras.activations.relu(hidden)
hidden = tf.keras.layers.Dense(1)(hidden)
outputs = tf.keras.activations.sigmoid(hidden)

model = tf.keras.Model(inputs=inputs, outputs=outputs, name='Model')

In [10]:
model.summary()

Model: "Model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 8)]               0         
                                                                 
 dense_4 (Dense)             (None, 10)                90        
                                                                 
 tf.nn.relu (TFOpLambda)     (None, 10)                0         
                                                                 
 dense_5 (Dense)             (None, 1)                 11        
                                                                 
 tf.math.sigmoid (TFOpLambda  (None, 1)                0         
 )                                                               
                                                                 
Total params: 101
Trainable params: 101
Non-trainable params: 0
_______________________________________________________________

In [11]:
tf.keras.utils.plot_model(model)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model/model_to_dot to work.


### Model compilation and training

In [12]:
# compile the model with selected optimizer, loss and metrics
model.compile(
        optimizer=tf.optimizers.Adam(), # Several other possibilities for optimizers 
        loss=tf.losses.BinaryCrossentropy(), # Select the proper loss for the task
        metrics=[tf.keras.metrics.AUC(), tf.keras.metrics.BinaryAccuracy()], # Select the proper metrics for the task
)

In [13]:
print('\n>>> Bias of the last layers:')
print(model.layers[3].weights[1].numpy())

print('\n>>> Kernel of the last layers:')
print(model.layers[3].weights[0].numpy())

print('\n>>> Bias of the first layers:')
print(model.layers[1].weights[1].numpy())

print('\n>>> Kernel of the first layers:')
print(model.layers[1].weights[0].numpy())


>>> Bias of the last layers:
[0.]

>>> Kernel of the last layers:
[[-0.16962433]
 [ 0.53847915]
 [ 0.22006738]
 [-0.33006412]
 [ 0.02976835]
 [ 0.55226606]
 [ 0.67009264]
 [-0.22814786]
 [ 0.1265648 ]
 [-0.71368927]]

>>> Bias of the first layers:
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]

>>> Kernel of the first layers:
[[-0.28134605  0.56252575 -0.57494134  0.23360962 -0.10974255  0.4265374
  -0.37385345  0.32267743  0.03222656  0.5710387 ]
 [-0.4842928   0.57292736  0.21615356 -0.04111856 -0.13085258 -0.4341614
   0.5598422   0.5346494  -0.42215216 -0.3225607 ]
 [-0.36994952  0.23800689  0.4745934   0.55855095  0.12056136 -0.5526602
  -0.56815046  0.3487463   0.33714277 -0.1901817 ]
 [-0.11928508  0.3998114   0.02599156 -0.5677908  -0.21454743 -0.21722722
   0.28900826 -0.54289865  0.44407773 -0.31914502]
 [ 0.3518539   0.5277877   0.14496279 -0.2784478   0.4090321   0.14122528
   0.18519694 -0.41088268 -0.2665919   0.4196201 ]
 [ 0.03714979 -0.21809056 -0.06588447  0.01100039  0.07349062 -0

In [14]:
# train the model with default setting
model.fit(X_train, y_train, batch_size=64, epochs=100)

Epoch 1/100
200/200 [==============================] - 1s 2ms/step - loss: 0.7701 - auc: 0.5412 - binary_accuracy: 0.5362
Epoch 2/100
200/200 [==============================] - 0s 2ms/step - loss: 0.5502 - auc: 0.8134 - binary_accuracy: 0.7320
Epoch 3/100
200/200 [==============================] - 0s 2ms/step - loss: 0.4535 - auc: 0.8819 - binary_accuracy: 0.8005
Epoch 4/100
200/200 [==============================] - 0s 2ms/step - loss: 0.4064 - auc: 0.9037 - binary_accuracy: 0.8258
Epoch 5/100
200/200 [==============================] - 0s 2ms/step - loss: 0.3766 - auc: 0.9175 - binary_accuracy: 0.8431
Epoch 6/100
200/200 [==============================] - 0s 2ms/step - loss: 0.3537 - auc: 0.9278 - binary_accuracy: 0.8548
Epoch 7/100
200/200 [==============================] - 0s 2ms/step - loss: 0.3342 - auc: 0.9357 - binary_accuracy: 0.8653
Epoch 8/100
200/200 [==============================] - 0s 2ms/step - loss: 0.3178 - auc: 0.9418 - binary_accuracy: 0.8750
Epoch 9/100
200/200 [===

In [15]:
# Evaluate the model and predict for the test data
model.evaluate(X_test, y_test)
test_pred = model.predict(X_test)

125/125 [==============================] - 1s 2ms/step - loss: 0.1985 - auc: 0.9775 - binary_accuracy: 0.9335


In [16]:
for pred, true in zip(test_pred, y_test[0:10]):
    print('{} - {:.2f}'.format(true, pred[0]))

1 - 0.28
0 - 0.28
0 - 0.31
1 - 1.00
0 - 0.00
0 - 0.03
1 - 0.96
0 - 0.01
1 - 0.95
1 - 1.00


### Add early stopping and regularization

In [17]:
# Input layer
inputs = tf.keras.Input(shape=(X_train.shape[1]))

# Hidden layer with regularization and ReLU
hidden = tf.keras.layers.Dense(10, kernel_regularizer=tf.keras.regularizers.l2(0.001))(inputs)
hidden = tf.keras.activations.relu(hidden)

# Output layer with regularization and sigmoid
outputs = tf.keras.layers.Dense(1, kernel_regularizer=tf.keras.regularizers.l2(0.001))(hidden)
outputs = tf.keras.activations.sigmoid(outputs)

model = tf.keras.Model(inputs=inputs, outputs=outputs, name='RegularizedModel')

model.compile(
        optimizer=tf.optimizers.Adam(),
        loss=tf.losses.BinaryCrossentropy(),
        metrics=[tf.keras.metrics.AUC(), tf.keras.metrics.BinaryAccuracy()],
)

model.summary()

Model: "RegularizedModel"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 8)]               0         
                                                                 
 dense_6 (Dense)             (None, 10)                90        
                                                                 
 tf.nn.relu_1 (TFOpLambda)   (None, 10)                0         
                                                                 
 dense_7 (Dense)             (None, 1)                 11        
                                                                 
 tf.math.sigmoid_1 (TFOpLamb  (None, 1)                0         
 da)                                                             
                                                                 
Total params: 101
Trainable params: 101
Non-trainable params: 0
____________________________________________________

In [18]:
epochs = 200

early_call = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=10, restore_best_weights=True
)

model.fit(X_train,
          y_train,
          epochs=epochs,
          validation_data=(X_dev, y_dev),
          callbacks=[early_call])

Epoch 1/200
400/400 [==============================] - 2s 3ms/step - loss: 0.5817 - auc_1: 0.7856 - binary_accuracy: 0.7073 - val_loss: 0.4627 - val_auc_1: 0.8939 - val_binary_accuracy: 0.8109
Epoch 2/200
400/400 [==============================] - 1s 2ms/step - loss: 0.4184 - auc_1: 0.9092 - binary_accuracy: 0.8338 - val_loss: 0.3692 - val_auc_1: 0.9318 - val_binary_accuracy: 0.8541
Epoch 3/200
400/400 [==============================] - 1s 2ms/step - loss: 0.3488 - auc_1: 0.9384 - binary_accuracy: 0.8700 - val_loss: 0.3235 - val_auc_1: 0.9473 - val_binary_accuracy: 0.8828
Epoch 4/200
400/400 [==============================] - 1s 2ms/step - loss: 0.3158 - auc_1: 0.9496 - binary_accuracy: 0.8877 - val_loss: 0.3032 - val_auc_1: 0.9533 - val_binary_accuracy: 0.8950
Epoch 5/200
400/400 [==============================] - 1s 2ms/step - loss: 0.3001 - auc_1: 0.9542 - binary_accuracy: 0.8931 - val_loss: 0.2923 - val_auc_1: 0.9574 - val_binary_accuracy: 0.8941
Epoch 6/200
400/400 [==============

### Gridsearch and Tensorboard
Run gridsearch over hidden layer size, L2 regularization, activation, check the outputs in Tensorboard

I recommend not to run Tensorboard from Jupyter notebook but from terminal directly

use "tensorboard --logdir logs" in command line

In [19]:
# hidden_sizes = [2, 5, 10, 20, 50]
# l2_regs = [0.01, 0.001, 0.0001]
# activations = ['relu', 'tanh']

hidden_sizes = [2, 5]
l2_regs = [0.01]
activations = ['relu', 'tanh']

epochs = 10
batch_size = 64

early_call = tf.keras.callbacks.EarlyStopping(monitor='val_AUC', mode='max', patience=10, restore_best_weights=True)

for activation in activations:
    for l2_reg in l2_regs:
        for hidden_size in hidden_sizes:
            if activation == 'relu':
                activate = tf.keras.activations.relu
            elif activation == 'tanh':
                activate = tf.keras.activations.tanh

            # Create Tensorboard Callback
            param_string = 'act-{},l2-{},hs-{}'.format(activation, l2_reg, hidden_size)
            log_dir = 'logs/binary_classification_test/' + param_string
            tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

            # Input layer
            inputs = tf.keras.Input(shape=(X_train.shape[1]))

            # Hidden layer with regularization and ReLU
            hidden = tf.keras.layers.Dense(hidden_size, kernel_regularizer=tf.keras.regularizers.l2(l2_reg))(inputs)
            hidden = activate(hidden)

            # Output layer with regularization and sigmoid
            outputs = tf.keras.layers.Dense(1, kernel_regularizer=tf.keras.regularizers.l2(l2_reg))(hidden)
            outputs = tf.keras.activations.sigmoid(outputs)

            model = tf.keras.Model(inputs=inputs, outputs=outputs, name='RegularizedModel')

            model.compile(
                    optimizer=tf.optimizers.Adam(),
                    loss=tf.losses.BinaryCrossentropy(),
                    metrics=[tf.keras.metrics.AUC(name='AUC'), tf.keras.metrics.BinaryAccuracy()],
            )

            # Train the model
            model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs,
                      validation_data=(X_dev, y_dev),
                      callbacks=[early_call, tensorboard_callback])

Epoch 1/10
200/200 [==============================] - 2s 4ms/step - loss: 0.6801 - AUC: 0.7464 - binary_accuracy: 0.6304 - val_loss: 0.6487 - val_AUC: 0.7993 - val_binary_accuracy: 0.6966
Epoch 2/10
200/200 [==============================] - 0s 2ms/step - loss: 0.6180 - AUC: 0.8145 - binary_accuracy: 0.7193 - val_loss: 0.5824 - val_AUC: 0.8358 - val_binary_accuracy: 0.7575
Epoch 3/10
200/200 [==============================] - 1s 3ms/step - loss: 0.5646 - AUC: 0.8314 - binary_accuracy: 0.7566 - val_loss: 0.5413 - val_AUC: 0.8431 - val_binary_accuracy: 0.7862
Epoch 4/10
200/200 [==============================] - 0s 2ms/step - loss: 0.5368 - AUC: 0.8343 - binary_accuracy: 0.7730 - val_loss: 0.5215 - val_AUC: 0.8445 - val_binary_accuracy: 0.7984
Epoch 5/10
200/200 [==============================] - 0s 2ms/step - loss: 0.5231 - AUC: 0.8377 - binary_accuracy: 0.7802 - val_loss: 0.5113 - val_AUC: 0.8436 - val_binary_accuracy: 0.8019
Epoch 6/10
200/200 [==============================] - 1s 3ms